<a href="https://colab.research.google.com/github/aralzaabi1991-coder/Training-ENEC/blob/main/Weather_LLM_wrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai==0.28 requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 2.9.0
    Uninstalling openai-2.9.0:
      Successfully uninstalled openai-2.9.0


In [2]:
import os
from google.colab import userdata

import openai
# Retrieve API keys from Colab's secure storage
openai_api_key = userdata.get("openai")

# Set them as environment variables

if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key

In [3]:
import requests

def get_current_weather(location, unit='celsius'):
    weather_api_key = "a7b109315b6ced6a9cd5177bdb98ca82"
    base_url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={weather_api_key}&units=metric"
    response = requests.get(base_url)
    data = response.json()

    weather_description = data['weather'][0]['description']
    return {
        "location": location,
        "temperature": data['main']['temp'],
        "weather": weather_description
    }

In [4]:
print(get_current_weather("Dubai"))

{'location': 'Dubai', 'temperature': 28.96, 'weather': 'broken clouds'}


In [5]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city, e.g. San Francisco"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"]
                }
            },
            "required": ["location"]
        }
    }
]

In [6]:
functions

[{'name': 'get_current_weather',
  'description': 'Get the current weather in a given location',
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string',
     'description': 'The city, e.g. San Francisco'},
    'unit': {'type': 'string', 'enum': ['celsius', 'fahrenheit']}},
   'required': ['location']}}]

In [12]:
#Function Definition and Initial Message Handling
def weather_chat(user_message):
    messages=[]
    messages.append({"role": "user", "content": user_message})
    messages.append({"role": "assistant", "content": "You are a weather bot . Answer only in Celsius."})



        # Sending Initial Message to OpenAI
    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature = 0,
            max_tokens=256,
            top_p=0.5,
            frequency_penalty=0,
            presence_penalty=0,
            messages=messages,
            functions=functions
        )

    #Handling Function Calls and Fetching Weather Data
    try:
        function_call = response['choices'][0]['message']['function_call']
        arguments = eval(function_call['arguments'])

        # Fetch weather data using the extracted arguments
        weather_data = get_current_weather(arguments['location'])

        # Append the function call and weather data to the messages
        messages.append({"role": "assistant", "content": None, "function_call": {"name": "get_current_weather", "arguments": str(arguments)}})
        messages.append({"role": "function", "name": "get_current_weather", "content": str(weather_data)})

#magic of llm
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages
        )

        return response['choices'][0]['message']['content']
    except Exception as e:
        return "shut-up"



In [13]:
import openai
openai.api_key = openai_api_key
weather_chat("is it hot in DUbaii")

'The current temperature in Dubai is 28.96 degrees Celsius, and it is partly cloudy.'

In [14]:
#qc
weather_chat("كم درجة الحرارة في الشارقة؟ ")

'درجة الحرارة في الشارقة حالياً هي 29.16 درجة مئوية مع وجود بعض السحب المتفرقة.'

In [10]:
!pip install gradio

In [15]:
# Define Gradio interface
import gradio as gr
iface = gr.Interface(
    fn=weather_chat,
    inputs=gr.Textbox(label="Weather Queries"),
    outputs=gr.Textbox(label="Weather Updates"),
    title = "Weather Bot",
    description = "Ask me anything about weather!"
)

# Launch the Gradio interface
iface.launch(share="True")

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://aade5ad65f0016ed1e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
